## 1 - Simple Sentiment Analysis

Trong series này, chúng ta sẽ xây dựng một mô hình học máy phân biệt cảm xúc (ví dụ: phát hiện xem một câu bình luận là tiêu cực (neg) hay tích cực (pos)) sử dụng pytorch và torchtext. Tập dữ liệu sử dụng ở đây là: [IMDb Dataset](http://ai.stanford.edu/~amaas/data/sentiment/).

Trong Notebook đầu tiên này, chúng ta khởi động bằng việc đọc hiểu các khái niệm chung, chưa cần quan tâm quá nhiều đến kết quả. Các Notebook sau sẽ xây dựng dựa trên những kiến thức nền tảng này và cải thiện kết quả.

### Introduction

Chúng ta sẽ sử dụng **recurrent neural network** (RNN) bởi vì đây là phương án phổ biến trong bài toán phân tích chuỗi (analysing sequences). Một RNN nhận đầu vào là một chuỗi các từ, X = {$x_1$, ..., $x_T$} mỗi lần, và sinh ra một **_hidden state: h_**, cho mỗi từ. Chúng ta sử dụng RNN một cách tuần tự bằng cách đưa từ hiện tại $x_t$ và **trạng thái ẩn** của từ phía trước, $h_{t-1}$ để sinh ra **trạng thái ẩn tiếp theo: $h_t$** <br>
* $h_t$ = RNN($x_t$ $h_{t-1}$) <br>

Sau khi thu được trạng thái ẩn cuối cùng: $h_T$ (bằng cách đưa từ cuối cùng của câu $x_T$ vào RNN) chúng ta đưa nó qua một linear layer f (hay còn gọi là fully connected layer), để cho ra kết quả dự đoán, $\hat{y}$ = f($h_T$). <br>

Example:

![RNN + Linear layer](./image/1_rnn_figure1.PNG "RNN + Linear layer")

**Note**:
* Màu vàng là RNN layer, màu trắng là Linear layer.
* Ta sử dụng cùng một RNN cho tất cả các từ, do đó chúng có cùng tham số (hoặc gọi là chia sẻ tham số).
* Trạng thái ẩn $h_0$ có dạng tensor, được khởi tạo bằng 0.

### Preparing Data

Một trong những khái niệm cơ sở trong TorchText là ***Field***. Nó định nghĩa các mà dữ liệu được xử lý. Trong nhiệm vụ phân tích cảm xúc, dữ liệu bao gồm phần đánh giá và phần nhãn (pos hoặc neg) đều ở dạng chuỗi văn bản. <br>
Các tham số trong ***Field*** định nghĩa cách mà dữ liệu được xử lý. <br>
Chúng ta sử dụng trường ***TEXT*** để định nghĩa cách xử lý phần đánh giá và trường ***LABEL*** xác định cách xử lý phần nhãn. <br>
Trường ***TEXT*** có tham số ***tokenize='spacy'*** định nghĩa "tokenization" (tokenization là việc chia chuỗi văn bản thành các "tokens" riêng biệt) được xử lý bằng tokenizer trong thư viện [spaCy](https://spacy.io/). Nếu không truyền đối số tokenize, chuỗi sẽ mặc định được chia thành các tokens, ngăn cách bằng dấu cách (space). Bên cạnh đó, chúng ta cũng cần định nghĩa đối số ***tokenizer_language***, giúp thông báo torchtext sẽ sử dụng thư viện ngông ngữ nào. <br>
***LABEL*** được định nghĩa bởi ***LabelField***, một trường con của ***Field***.

In [16]:
import spacy
spacy.load('en_core_web_sm')

In [17]:
import torch
from torchtext.legacy import data

SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                    tokenizer_language = 'en_core_web_sm')

LABEL = data.LabelField(dtype = torch.float)

1. Download dataset IMDb, gồm 50.000 review phim.
2. Do tập dữ liệu chỉ có train và test, ta cần chia thêm tập validation từ tập train.

In [18]:
from torchtext.legacy import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

In [19]:
print("Number of training examples: ", len(train_data))
print("Number of testing examples: ", len(test_data))

Number of training examples:  25000
Number of testing examples:  25000


In [20]:
print(vars(train_data.examples[0]))

{'text': ['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '"', 'Teachers', '"', '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"', 'Teachers', '"', '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High', '.', 'A', 'classic', 'l

In [21]:
import random
# 80% train, 20% validate
train_data, valid_data = train_data.split(random_state = random.seed(SEED), split_ratio = 0.8)

In [22]:
print("Number of training examples: ", len(train_data))
print("Number of validation examples: ", len(valid_data))
print("Number of testing examples: ", len(test_data))

Number of training examples:  20000
Number of validation examples:  5000
Number of testing examples:  25000


Tiếp theo, chúng ta sẽ xây dựng ***từ điển*** (_vocabulary_). <br>
Từ điển là một bảng tra cứu (lookup table), mỗi từ trong tập dữ liệu sẽ có một _chỉ số_ trong bảng. <br>
> Tại sao phải xây dựng từ điển? <br>
=> Câu trả lời là các mô hình học máy chỉ có thể làm việc với kiểu dữ liệu số, không thể làm việc với các kiểu dữ liệu khác. Mỗi một _chỉ số_ trong bảng tra cứu tạo nên một _one-hot vector_ cho mỗi từ tương ứng. Một _one-hot vector_ là vector có các phần từ bằng 0, duy nhất có một phần từ bằng 1 và có kích thước bằng với kích thước của từ điển - ký hiệu là **V** (kích thước từ điển là số unique word có trong từ điển). <br>
Ví dụ:

![One hot vector example](./image/1_onehot_vector_example.PNG)

Như ví dụ trên, kích thước từ điển bằng 4, do đó kích thước của one-hot vector là 4.

> Tuy nhiên, số lượng unique words trong tập train khoảng 100.000 từ => one-hot vector sẽ có kích thước là 100.000 chiều! Số lượng lớn như vậy khiến quá trình huấn luyện rất chậm, đông thời không thể nạp hết vào GPU.<br>
Có 2 cách để giảm bợt kích thước từ điển:
> 1. Chỉ lấy n từ đầu tiên trong từ điển.
> 2. Lấy m từ cuối cùng trong từ điển. <br>
> Chúng ta sẽ sử dụng cách đầu tiên, giữ 25.000 từ đầu tiên trong vocabulary.

In [23]:
MAX_VOCAB_SIZE = 25_000
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

> Có một câu hỏi: Tại sao chúng ta chỉ xây dựng Vocab trên tập train? <br>
=> Khi test, chúng ta không muốn mô hình biết trước các từ có trong tập test, như vậy tính khách quan của mô hình sẽ không còn.

In [24]:
print("Unique tokens in TEXT vocabulary: ", len(TEXT.vocab))
print("Unique tokens in LABEL vocabulary: ", len(LABEL.vocab))

Unique tokens in TEXT vocabulary:  25002
Unique tokens in LABEL vocabulary:  2


> Kích thước từ điển là 25002 thay vì 25000 là do có token đệm (pad). <br>
Khi ta truyền các câu vào mô hình, ta sẽ truyền theo từng batch, tất cả các câu phải có cùng kích thước, các câu ***ngắn hơn câu dài nhất trong từ điển*** đều được đệm sao cho các thành phần trong batch có cùng kích thước. <br>
Ví dụ:

![Padding example](./image/1_padsentence_example.PNG)

In [25]:
TEXT.vocab.freqs.most_common(20)

[('the', 232316),
 (',', 219807),
 ('.', 189207),
 ('and', 125132),
 ('a', 124608),
 ('of', 115066),
 ('to', 107184),
 ('is', 87276),
 ('in', 70016),
 ('I', 61807),
 ('it', 61373),
 ('that', 56356),
 ('"', 50562),
 ("'s", 49467),
 ('this', 48446),
 ('-', 42132),
 ('/><br', 40779),
 ('was', 40081),
 ('as', 34777),
 ('with', 34057)]

In [26]:
TEXT.vocab.itos[:10]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']

In [27]:
LABEL.vocab.stoi

defaultdict(None, {'neg': 0, 'pos': 1})

> Bước cuối cùng trong việc chuẩn bị dữ liệu là tạo iterators. Iterators cho phép lặp xuyên suốt training/evaluation loop, và trả về batch (đã được chuyển về dạng tensor) ở mỗi lần lặp. <br>
Chúng ta sẽ sử dụng ***BucketIterator***, _trả về một batch các sentences có cùng kích thước_, giúp **tối thiểu hóa việc pading sentences**:

In [28]:
BATCH_SIZE = 64


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator,\
valid_iterator,\
test_iterator = data.BucketIterator.splits((train_data, valid_data, test_data),                
                                            batch_size = BATCH_SIZE,
                                            device = device)

### Build the Model

> Mô hình gồm 3 layer: _embedding_ layer, _RNN_ layer và _linear_ layer. Các layer có tham số được khởi tạo ngẫu nhiên nếu không được chỉ định ban đầu. <br>
_Embedding_ layer được sử dụng để transform các one-hot vector ở dạng thưa thành các dense embedding vector (dense có nghĩa là vector có chiều nhỏ hơn và các phần tử là các số thực). Embedding layer thực chất là một lớp fully connected. Bên cạnh việc giảm chiều dữ liệu cho RNN, một số lý thuyết chỉ ra rằng các từ giống nhau trong phần nhận xét sẽ được ánh xạ gần nhau trong không gian dense vector. Đọc thêm về word embedding tại đây: [Here](https://monkeylearn.com/blog/word-embeddings-transform-text-numbers/). <br>
RNN layer tương tự như RNN mô tả phía trên. <br>
Cuối cùng là lớp linear transform hidden state thành đầu ra có số chiều phù hợp thông qua một lớp fully connected f($h_T$). <br>
Phương thức **forward** được gọi khi ta truyền vào một example point. <br>
Mỗi một input batch được truyền qua một embedding layer để thu được **embedded**, là một dense vector biểu diễn câu ta truyền vào. **embedded** là một tensor có kích thước **[sentence length, batch size, embedding dim]**. <br>
Sau đó **embedded** được truyền vào RNN layer. <br>
RNN trả về 2 tensors:
1. ***output*** có kích thước **[sentence length, batch size, embedding dim]**.
2. ***hidden*** có kích thước **[1, batch size, embedding dim]**.
> ***output*** là sự kết hợp của các hidden state qua các bước, trong khi ***hidden*** đơn giản là hidden state cuối cùng.
3. Cuối cùng, ta truyền ***hidden*** vào lớp linear để sinh ra kết quả.

![Padding example](./image/1_rnn_fc_model.PNG)

In [29]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, text):
        # text = [sent len, batch size]
        embedded = self.embedding(text)

        # embedded = [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)

        # output = [sent len, batch size, hid dim]
        # hidden = [1, batch size, hid dim]

        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        return self.fc(hidden.squeeze(0))

* input_dim là kích thước của one-hot vectors, bằng vocabulary size.
* embedding_dim là kích thước của dense word vectors. Thường nằm trong khoảng 50 - 250, tuy nhiên nó còn phụ thuộc vào kích thước vocabulary.
* hidden_dim là kích thước của hidden states, thường nằm trong khoảng 100 - 500, tuy nhiên nó còn phụ thuộc vào kích thước vocabulary, kích thước của dense vector và độ phức tạp của task đang thực hiện.
* output_dim là kích thước đầu ra, trong trường hợp này là 0 hoặc 1, 2 class biểu thị cho 2 nhãn (neg và pos).

In [30]:
input_dim = len(TEXT.vocab)
embedding_dim = 100
hidden_dim = 256
output_dim = 1

model_simple = RNN(input_dim, embedding_dim, hidden_dim, output_dim)

In [31]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model_simple):,} trainable parameters')

The model has 2,592,105 trainable parameters


### Train the Model

> Đầu tiên, ta cần tạo một optimizer. Đây bản chất là một thuật toán giúp cập nhật tham số cho mô hình.

In [32]:
optimizer = torch.optim.SGD(model_simple.parameters(), lr=1e-3)

> Tiếp theo, ta cần định nghĩa hàm loss (trong pytorch thường gọi là criterion). <br>
Do đầu ra là 0 hoặc 1, ta sử dụng Binary cross entropy.

In [33]:
criterion = nn.BCEWithLogitsLoss()

In [34]:
model_simple = model_simple.to(device)
criterion = criterion.to(device)

In [35]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

> * Hàm ***train*** lặp xuyên suốt training set, mỗi một batch một lần. <br>
***model.train()*** đưa mô hình vào chế độ huấn luyện - bật _drop out_ và _batch normalization_. <br>
> * Với mỗi batch training, đầu tiên ta cần đưa gradients về 0. Mỗi tham số của mô hình có thuộc tính ***grad*** lưu trữ gradient được tính bởi ***creterion***. Pytorch không tự động đưa gradient về 0, do đó ta cần thực hiện. <br>
> * Lưu ý, khi đưa một batch các câu, ***batch.text*** vào mô hình, ta không cần dùng phương thức ***forward***. ***squeeze*** cần thiết bởi việc dự đoán được khởi tạo với kích thước ***[batch size, 1]***, và ta cần xóa bỏ chiều 1, predictions yêu cầu đầu vào có kích thước ***[batch size]***. <br>
> * Loss và accuracy được tính toán bằng cách sử dụng đầu ra của mô hình và label của batch - ***batch.label***, loss được tính trung bình trên toàn batch. <br>
> * Ta tính gradient của mỗi tham số bằng ***loss.backward()***, sau đó cập nhật gradient bằng ***optimizer.step()***.

In [36]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

> * ***evaluate*** khá giống với ***train***, tuy nhiên ta cần điều chỉnh một chút.
> * ***model.eval()*** đưa mô hình vào chế độ đánh giá - tắt dropout và batch normalization.
> * Quá trình evaluate không cần tính đạo hàm => sử dụng ***with torch.no_grad()*** để giảm bộ nhớ.

In [37]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [38]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [40]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model_simple, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model_simple, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model_simple.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

KeyboardInterrupt: 

In [45]:
torch.save(model_simple.state_dict(), 'tut1-model.pt')

In [46]:
model_simple.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model_simple, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.710 | Test Acc: 47.25%


### Next Steps

> * Gói các câu đã được đệm lại cùng với nhau
> * Sử dụng pre-train word embeddings.
> * Sử dụng kiến trúc RNN khác.
> * Sử dụng bidirectional RNN.
> * Sử dụng RNN nhiều lớp (multi-layer RNN).
> * Regularization.
> * Sử dụng optimizer khác.

## END